In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoProcessor, SeamlessM4TModel

phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/Users/mawadalla/anaconda3/envs/v2/lib/python3.12/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
sentence = "It's raining cats and dogs."
src_lang = "eng"
tgt_lang = "arb"

In [3]:
messages = [
        {"role": "user", "content": f"Given the sentence: '{sentence}', please either output the sentence as is or change any figurative language to its literal meaning and then output the modified sentence."},
    ]
phi_pipe = pipeline(
    "text-generation",
    model=phi_model,
    tokenizer=phi_tokenizer,
)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}
phi_output = phi_pipe(messages, **generation_args)
modified_sentence = phi_output[0]['generated_text']

del phi_pipe
del messages
del phi_model
del phi_tokenizer
del phi_output

print(f'Original sentence: {sentence}')
print(f'Modified sentence: {modified_sentence}')

/Users/mawadalla/anaconda3/envs/v2/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


KeyboardInterrupt: 

In [ ]:
seamless_processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-large")
seamless_model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-large")



In [ ]:
text_inputs = seamless_processor(text=modified_sentence, return_tensors="pt")
output_tokens = seamless_model.generate(**text_inputs, tgt_lang=tgt_lang, generate_speech=False)
translated_text = seamless_processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

In [ ]:
print(f'Translated sentence: {translated_text}')